## 7.8.1 


[링크 텍스트](https://wikidocs.net/44249)

In [1]:
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f782156f810>)

In [2]:
ls

ratings_test.txt  ratings_train.txt  sample_data/


In [3]:
import pandas as pd

df_train = pd.read_csv("ratings_train.txt", delimiter='\t',keep_default_na= False)
df_train.head(n=3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [4]:
text_train, y_train = df_train['document'].values, df_train['label'].values

In [5]:
df_test = pd.read_csv("ratings_test.txt", delimiter='\t',keep_default_na= False)
text_test = df_test['document'].values
y_test = df_test['label'].values

In [7]:
import numpy as np

len(text_train), np.bincount(y_train)

(150000, array([75173, 74827]))

In [8]:
len(text_test), np.bincount(y_test)

(50000, array([24827, 25173]))

In [9]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 62.3 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
     |████████████████████████████████| 448 kB 45.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [10]:
from konlpy.tag import Okt
okt_tag = Okt()

In [11]:
def okt_tokenizer(text):
    return okt_tag.morphs(text)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

okt_param_grid = {'tfidfvectorizer__min_df':[3,5,7],
                  'tfidfvectorizer__ngram_range':[(1,1),(1,2),(1,3)],
                  'logisticregression__C':[0.1,1,10]}
okt_pipe = make_pipeline(TfidfVectorizer(tokenizer=okt_tokenizer),
                         LogisticRegression(solver='liblinear'))

okt_grid =  GridSearchCV(okt_pipe, okt_param_grid, cv = 3) 

okt_grid.fit(text_train[0:1000],y_train[0:1000])
print("최상의 교차 검증 점수 : {:.3f}".format(okt_grid.best_score_))
print("최적의 교차 검증 매개변수 : ", okt_grid.best_params_)

최상의 교차 검증 점수 : 0.704
최적의 교차 검증 매개변수 :  {'logisticregression__C': 1, 'tfidfvectorizer__min_df': 3, 'tfidfvectorizer__ngram_range': (1, 1)}


In [13]:
X_test_okt = okt_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(text_test)
score = okt_grid.best_estimator_.named_steps['logisticregression'].score(X_test_okt, y_test)
print("텍스트 세트 점수 : {:.3f}".format(score))

텍스트 세트 점수 : 0.705


In [14]:
from konlpy.tag import Mecab
mecab = Mecab()
def mecab_tokenizer(text):
    return mecab.morphs(text)

Exception: ignored

In [ ]:
mecab_param_grid = {'tfidfvectorizer__min_df':[3,5,7],
                  'tfidfvectorizer__ngram_range':[(1,1),(1,2),(1,3)],
                  'logisticregression__C':[0.1,1,10,100]}
mecab_pipe = make_pipeline(TfidfVectorizer(tokenizer=mecab_tokenizer),
                         LogisticRegression(solver='liblinear'))

mecab_grid =  GridSearchCV(mecab_pipe, mecab_param_grid,n_jobs=1, cv = 3) 

mecab_grid.fit(text_train[0:1000],y_train[0:1000])
print("최상의 크로스 밸리데이션 점수 : {:.3f}".format(mecab_grid.best_score_))
print("최적의 크로스 밸리데이션 파라미터 : ", mecab_grid.best_params_)

In [ ]:
X_test_mecab = mecab_grid.best_estimator_.named_steps["tfidfvectorizer"].transform(text_test)
score = mecab_grid.best_estimator_.named_steps['logisticregression'].score(X_test_mecab, y_test)
print("텍스트 세트 점수 : {:.3f}".format(score))